# Digital epidemiology
### University of Trento
### AA 2024/2025

Author: Michele Tizzoni

---

## Notebook 5

### Epidemics on temporal networks 

**The temporal dynamics of networks affect dynamical processes defined one them**

In 2001, Liljeros and collaborators found that the network of human sexual contacts is "scale-free" concluding that STI can be hard to eradicate because of such property. 

See: https://www.nature.com/nature/journal/v411/n6840/full/411907a0.html

However, their conclusions were only partially true because the network of human sexual contacts is a *dynamical network* and temporal fluctuations are very important when considering a dynamical processes that take place on top of it.

In 2011, Rocha and collaborators published a temporal dataset of sexual contacts in Brazil: http://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1001109.
We use this dataset to simulate an epidemic process (SI) and compare results between the static aggregated network and the dynamic network.

In [ ]:
import networkx as nx
from collections import defaultdict

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
filepath='/Users/micheletizzoni/Documents/GitHub/Digital_epidemiology/datasets/Dataset_sexual_network.csv'

In [ ]:
edgelist = defaultdict(list)

In [ ]:
finput=open(filepath,'r')
for line in finput.readlines():
    
    if line[0]!='#':#not a comment
        s=line.strip().split(';')
        
        day=int(s[2])
        edge=(int(s[0]), int(s[1]))
                
        edgelist[day].append(edge)
        
finput.close()

In [ ]:
len(edgelist)

We first create the aggregated network 

In [ ]:
G_agg=nx.Graph()
G_agg.disease_status={}

In [ ]:
daystart = 800#we skip the transient
dayend = 1800

In [ ]:
for d in edgelist:
    if d>=daystart and d<=dayend:
        links=edgelist[d]
        G_agg.add_edges_from(links)#we add the link to the graph

print("The aggregated network has", len(G_agg.nodes()),"nodes")
print("The aggregated network has", len(G_agg.edges()),"links")

The network is not connected

In [ ]:
nx.is_connected(G_agg)

In [ ]:
nx.number_connected_components(G_agg)

In [ ]:
for i in nx.connected_components(G_agg):
    print(len(i))

---
## Simulations on the time-aggregated graph

In [ ]:
#let's choose a random seed
import random

seed_links=edgelist[daystart]
print(len(seed_links))

random.shuffle(seed_links)
seed=seed_links[0][0]

print("The degree of the seed is", G_agg.degree(seed))

In [ ]:
infected_nodes = []
infected_nodes.append(seed)

for n in G_agg.nodes():
    if n in infected_nodes:
        G_agg.disease_status[n] = 1
        #infected
    else:
        G_agg.disease_status[n] = 0
        #susceptible

In [ ]:
Itime_agg = []

#there are 1000 days of network activity but the network does not change!

for t in range(0,1000):
    
    for i in infected_nodes:
        for j in G_agg.neighbors(i):
            if G_agg.disease_status[j] == 0: 
                G_agg.disease_status[j] = 1 #the probability of infection is 1!!
                
    #we cycle over the infected nodes
    infected_nodes=[]
    for n in G_agg.nodes():
        if G_agg.disease_status[n]==1:
            infected_nodes.append(n)
             
    Itime_agg.append(len(infected_nodes))

print("The final size of the epidemic is", float(len(infected_nodes))/len(G_agg.nodes()))

Let's visualize the time evolution of the infected nodes

In [ ]:
plt.figure(figsize=(10,7))
plt.xlabel('Time', fontsize=18)
plt.ylabel('Infected individuals', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.plot(range(0,len(Itime_agg)),Itime_agg)
plt.axis([0,20,-1,10600])

---
## Simulations on the full dynamic network

In [ ]:
G_dyn = nx.Graph()
G_dyn.disease_status = {}

In [ ]:
seed_links = edgelist[daystart]
random.shuffle(seed_links)
seed = seed_links[0][0]

In [ ]:
seed

In [ ]:
infected_nodes = []
infected_nodes.append(seed)

G_dyn.add_edges_from(seed_links)  # we consider only the links that are active on day 0

for n in G_dyn.nodes():
    if n in infected_nodes:
        G_dyn.disease_status[n] = 1
        # infected
    else:
        G_dyn.disease_status[n] = 0
        # susceptible

In [ ]:
Itime_dyn = []

for t in range(daystart, dayend + 1):

    links = edgelist[t]  # these are the links active on day t

    if t == daystart:
        print("The temporal network has", len(G_dyn.nodes()), "nodes on day", daystart)
        print("The temporal network has", len(G_dyn.edges()), "edges on day", daystart)
        print("The degree of the seed is", G_dyn.degree(seed))
    else:
        G_dyn.add_edges_from(links)
        for e in links:
            if e[0] not in G_dyn.disease_status:
                G_dyn.disease_status[e[0]] = 0
            if e[1] not in G_dyn.disease_status:
                G_dyn.disease_status[e[1]] = 0

    # transmission of the infection
    for i in infected_nodes:
        for j in G_dyn.neighbors(i):
            if G_dyn.disease_status[j] == 0:
                G_dyn.disease_status[j] = 1

    # we update the status of the infected nodes
    infected_nodes = []
    for n in G_dyn.nodes():
        if G_dyn.disease_status[n] == 1:
            infected_nodes.append(n)

    Itime_dyn.append(len(infected_nodes))

    # remove the active links of that day
    G_dyn.remove_edges_from(links)

#At the end of the epidemic we print the final attack rate of the epidemic
print(
    "The final size of the epidemic is", float(len(infected_nodes)) / len(G_dyn.nodes())
)

In [ ]:
plt.figure(figsize=(10,7))
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('time', fontsize=18)
plt.ylabel('infected individuals', fontsize=18)
plt.plot(range(0,len(Itime_agg)),Itime_agg, label='static')
plt.plot(range(0,len(Itime_dyn)),Itime_dyn, label='dynamic')
plt.legend()
#plt.axis([0,365,-1,10600])

As expected, the epidemic spreads much faster on the full aggregated network but is not realistic. Considering the full temporal activity slows down the diffusion and provides a more realistic description of the epidemic.